<a href="https://colab.research.google.com/github/sean-halpin/ml_tweepy_proj/blob/main/twitter_sentiment_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [2]:
BEARER_TOKEN = ""

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

In [37]:
API_KEY=""
API_SECRET=""
ACCESS_TOKEN=""
ACCESS_TOKEN_SECRET=""

In [47]:
import tweepy as tpy
import tweepy as tweepy
import pandas as pd

# auth=tpy.OAuthHandler(API_KEY,API_SECRET)
# auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
# api = tpy.API(auth)
client = tweepy.Client(
    bearer_token=BEARER_TOKEN
)

In [ ]:
# search_result=[]
# for tweet in tpy.Cursor(api.search_tweets, "english premier league -is:retweet",
#                               max_results=100).items(limit=200):
#   search_result.append(tweet)

In [48]:
# search_result = client.search_recent_tweets(query="english premier league -is:retweet",max_results=100)

In [52]:
search_result=[]
for tweet in tweepy.Paginator(client.search_recent_tweets, "english premier league -is:retweet",
                              max_results=100, tweet_fields=['created_at']).flatten(limit=200):
  search_result.append(tweet)

In [53]:
search_result[0].created_at

datetime.datetime(2022, 3, 30, 17, 18, tzinfo=datetime.timezone.utc)

In [ ]:
~list_of_lists = []
i=0
for sr in search_result:
  i+=1
  text = sr.text
  print(text)
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  list_of_lists.append([sr.text, str(labels[ranking[0]])])

In [ ]:
cols = ["tweet", "sentiment"]
df = pd.DataFrame(list_of_lists, columns=cols)
df.head()

In [ ]:
df['sentiment']. value_counts()

In [ ]:
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
df = df.replace(r'\r+|\n+|\t+',' ', regex=True)

In [ ]:
df.isnull().any()

In [ ]:
import time
epoch_time = int(time.time())

In [ ]:
df.to_csv("tweets_annotated.{}.csv".format(epoch_time), header=cols, index=False, escapechar="\\", encoding='utf-8')

In [ ]:
test_df = pd.read_csv("tweets_annotated.{}.csv".format(epoch_time))

In [ ]:
test_df.head(5)

In [ ]:
test_df['sentiment']. value_counts()